# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 03:23:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.49       Driver Version: 457.49       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   45C    P8     1W /  N/A |    410MiB /  4096MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2019년 SK Data**

In [5]:
lexicon_2019 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2019_sk.csv')

lexicon_2019.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,sk하이닉스,660,매일경제,2019010104,글로벌 불확실성·기업실적 둔화…`상저하고` 장세 전망 우세,http://news.mk.co.kr/newsRead.php?year=2019&no...,◆ 2019 경제기상도 / 증시 ◆ \r\n\n\n 주요 증권사 리서치센터에 따르면...,2019-01-02,4,61300,61400,60000,60600,1934295,0.001653,1,0,경제 기상도 증시 주요 증권사 리서치 센터 국내 증시 코스피 미만 저하 기록 전망 ...


In [6]:
lexicon_2019[lexicon_2019['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2019.dropna(axis=0, inplace=True)
lexicon_2019 = lexicon_2019.reset_index(drop=True)
lexicon_2019.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2019 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2019['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2019['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2019['Tokenization'][x].split())):
                    if lexicon_2019['Tokenization'][x].split()[w] == list(set(lexicon_2019['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2019['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2019['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2019 = pos_lexicon_2019.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2019['date'][x], positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z])))

***2019-01-02 Cosine Similarity between <강세> & <급락> : 0.507910966873169
***2019-01-02 Cosine Similarity between <개선> & <방안> : 0.5635414123535156
***2019-01-02 Cosine Similarity between <개선> & <개선> : 0.9999999403953552
***2019-01-02 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2019-01-02 Cosine Similarity between <상회한> & <견조할> : 0.5388216376304626
***2019-01-02 Cosine Similarity between <증대> & <저하> : 0.5609553456306458
***2019-01-02 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2019-01-02 Cosine Similarity between <추진> & <진행> : 0.5358567833900452
***2019-01-02 Cosine Similarity between <해결> & <문제> : 0.6186186075210571
***2019-01-02 Cosine Similarity between <해결> & <방안> : 0.5310750603675842
***2019-01-02 Cosine Similarity between <호황> & <업황> : 0.5544542670249939
***2019-01-02 Cosine Similarity between <호황> & <불황> : 0.7041272521018982
***2019-01-02 Cosine Similarity between <활성화> & <촉진> : 0.5009052157402039
***2019-01-02 Cosine Similarity between <증대> & <확보

In [11]:
pos_lexicon_2019

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2019-01-02,1,강세,급락,0.507911,1
1,2019-01-02,1,개선,방안,0.563541,1
2,2019-01-02,1,개선,개선,1.000000,1
3,2019-01-02,1,경신,기록,0.522991,2
4,2019-01-02,1,상회한,견조할,0.538822,1
...,...,...,...,...,...,...
13006,2020-01-02,2509,경신,기록,0.522991,2
13007,2020-01-02,2509,중요한,가장,0.537125,4
13008,2020-01-02,2509,추진,추진,1.000000,1
13009,2020-01-02,2509,추진,검토,0.511331,1


In [12]:
pos_lexicon_2019.to_csv('../../../../Code/Data/Test/Stock-Year/pos_sk_2019.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2019['news_word'])))

a

['프로그램매매',
 '구현',
 '최선',
 '패너마시티비치',
 '포시에트',
 '팽팽하',
 '사무특례',
 '추진',
 '우세',
 '흐트러지',
 '중인',
 '상쇄하',
 '격이므',
 '문제',
 '강학봉',
 '침체',
 '시너지',
 '활기',
 '수펙스',
 '창저우시',
 '웃돌',
 '텔레큼',
 '엠앤티삼강엠',
 '본격',
 '유데나필',
 '강세',
 '검토',
 '설진훈',
 '경의',
 '이미드',
 '롱숏펀드',
 '흐릿했다',
 '저하',
 '차정훈',
 '겹악재',
 '향상',
 '권종은',
 '최준우',
 '증진',
 '퍼부으',
 '어쩌자',
 '런정페이',
 '방법',
 '벳이글벳',
 '확보',
 '펀인터렉티브',
 '엠에스',
 '보통신',
 '소형모듈원전',
 '쎄미켐동진쎄미켐',
 '적극',
 '자공시',
 '호혜무역',
 '워너램버트',
 '전체수',
 '고기능고분자',
 '윈하이텍',
 '단점',
 '텍사트인스',
 '훈풍',
 '이점',
 '맥쿼리인프라',
 '중거리핵전력조약',
 '이권재',
 '방안',
 '프론텍',
 '계획',
 '의미',
 '사대부중',
 '조호윤',
 '젬백스젬백스',
 '발전',
 '기록',
 '케이에스',
 '개선',
 '확립',
 '이유섭',
 '업황',
 '증가',
 '랙티브',
 '쇼와덴코',
 '우호',
 '마련',
 '컨테이선',
 '고취',
 '브머티리얼스',
 '급증',
 '불황',
 '긴급물량',
 '차익',
 '메디콕스메디콕스',
 '가치나',
 '윈팩윈팩',
 '공유진',
 '코팅덕트',
 '띠웠다',
 '밑돌',
 '인폴딩',
 '루브리컨츠',
 '들보',
 '안중근함',
 '해법',
 '엠비덱스',
 '레론리맙',
 '정본부장',
 '언택트',
 '견조할',
 '박유악',
 '행복에프앤씨',
 '추동훈',
 '주요한',
 '밑돈',
 '비밀보호법',
 '응우옌비엣꽝',
 '문무역',
 '벽체',
 '중요',
 '급락',
 '전개',
 '엔드밀시장',
 '확대

In [14]:
b = list(set(list(pos_lexicon_2019[pos_lexicon_2019['cosine_similarity']>=0.7]['news_word'])))

b

['개선',
 '실패',
 '우월',
 '단점',
 '경의',
 '띠웠다',
 '약세',
 '최선',
 '밑돈',
 '불황',
 '셀루메드셀루메드',
 '장점',
 '충족',
 '쎄미켐동진쎄미켐',
 '추진']

In [15]:
c = list(set(list(pos_lexicon_2019[pos_lexicon_2019['cosine_similarity']>=0.65]['news_word'])))

c

['개선',
 '최선',
 '패너마시티비치',
 '증가',
 '추진',
 '우월',
 '실패',
 '불황',
 '셀루메드셀루메드',
 '쎄미켐동진쎄미켐',
 '적극',
 '메디콕스메디콕스',
 '감소',
 '단점',
 '웃돌',
 '약세',
 '띠웠다',
 '후루가',
 '셰도보팅',
 '장점',
 '충족',
 '경의',
 '밑돈']